# **Import Statments**

In [3]:
import pandas as pd 
from sklearn.neighbors import NearestNeighbors

# **Read In Data**

In [2]:
spotify = pd.read_csv('https://raw.githubusercontent.com/robinhester/ds-1/master/Data/songs1.csv')

In [3]:
spotify.head()

,Unnamed: 0,index,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity,genre
0,0,0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.00582,0.743,238373,0.339,0.000000,1,0.0812,-7.678,1,0.409,203.927,4,0.118,15,hip-hop
1,1,1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.02440,0.846,214800,0.557,0.000000,8,0.2860,-7.259,1,0.457,159.009,4,0.371,0,hip-hop
2,2,2,YG,1pWeQDyW5ieMmfjr8HP5TU,Suu Whoop,0.03260,0.799,185200,0.325,0.000035,2,0.1140,-9.910,1,0.506,157.966,4,0.233,3,hip-hop
3,3,3,YG,75NGdzR6LSC1r0KB5mltP9,Im A Thug,0.11300,0.775,204173,0.485,0.000000,9,0.0525,-5.817,1,0.128,90.995,4,0.464,38,hip-hop
4,4,4,YG,3AT4akM1Oj1dfCb0ZA9cdy,BOMPTOWN FINEST,0.24500,0.636,323613,0.577,0.000000,0,0.0778,-10.059,0,0.525,94.356,4,0.188,52,hip-hop


# **Engineering**

In [4]:
"""
Creating a copy of original data, deleting high cardinality and unrelated information.
Setting Index to track_id which is specifically related to the song and how backend 
will be pulling the data from API. Don't need it in the model, but still need it linked
to the information. Using pd.get_dummies for One Hot Encoding of just the genre 
column, then addding it back into the data, as well as deleting the original genre 
column. Concating the data back together and filling NaN's with zero, as it has not impact
as a zero. 
"""
train = spotify.copy()

cols_to_delete = ['track_name', 'artist_name', 'index']

train = train.drop(columns=cols_to_delete).set_index('track_id')
df = pd.get_dummies(train['genre'], prefix='Genre')

columns = ['genre','Unnamed: 0']
train = train.drop(columns=columns)
train = pd.concat([train, df], axis=1)
# train = train.drop(columns='Unnamed: 0')
train = train.fillna(0)

In [5]:
train.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity,Genre_acoustic,Genre_all,Genre_alternative,Genre_alternative rock,Genre_ambient,Genre_american,Genre_beats,Genre_black metal,Genre_blues,Genre_bollywood,Genre_british,Genre_canadian,Genre_chillout,Genre_chinese,Genre_christian,Genre_classical,Genre_contemporary classical,Genre_country,Genre_dance,Genre_dancehall,Genre_deep house,Genre_dream pop,Genre_drum and bass,Genre_dubstep,Genre_electronic,Genre_emo,...,Genre_house,Genre_indian,Genre_indie,Genre_indie pop,Genre_indie rock,Genre_instrumental,Genre_j-pop,Genre_japanese,Genre_jazz,Genre_k-pop,Genre_korean,Genre_latin,Genre_lo-fi,Genre_metalcore,Genre_minimal,Genre_piano,Genre_pop,Genre_pop punk,Genre_post-hardcore,Genre_post-rock,Genre_progressive house,Genre_progressive metal,Genre_progressive rock,Genre_punk,Genre_punk rock,Genre_rap,Genre_reggae,Genre_reggaeton,Genre_rnb,Genre_rock,Genre_shoegaze,Genre_singer-songwriter,Genre_soul,Genre_soundtrack,Genre_swedish,Genre_synthpop,Genre_techno,Genre_thrash metal,Genre_trance,Genre_trap
track_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2RM4jf1Xa9zPgMGRDiht8O,0.00582,0.743,238373,0.339,0.000000,1,0.0812,-7.678,1,0.409,203.927,4,0.118,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1tHDG53xJNGsItRA3vfVgs,0.02440,0.846,214800,0.557,0.000000,8,0.2860,-7.259,1,0.457,159.009,4,0.371,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1pWeQDyW5ieMmfjr8HP5TU,0.03260,0.799,185200,0.325,0.000035,2,0.1140,-9.910,1,0.506,157.966,4,0.233,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
75NGdzR6LSC1r0KB5mltP9,0.11300,0.775,204173,0.485,0.000000,9,0.0525,-5.817,1,0.128,90.995,4,0.464,38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3AT4akM1Oj1dfCb0ZA9cdy,0.24500,0.636,323613,0.577,0.000000,0,0.0778,-10.059,0,0.525,94.356,4,0.188,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **Nearest Neighbors Model**

In [6]:
"""
Creating a NearestNeighbor model, using kd_tree and 15 neighbors. Fitting that 
information to the train data, and printing the array resulting from the model.
My example, the song in row 8, is similar to the song in row 130489. Looking at
the information, it's comparing a rap and hip hop song, and are fairly similiar.
"""

nn = NearestNeighbors(n_neighbors=15, algorithm='kd_tree')
nn.fit(train)

print(nn.kneighbors([train.iloc[8].values]))
print("----------------------------")

(array([[ 0.        ,  9.99864532, 17.60883743, 20.08739426, 22.51827391,
        22.72016311, 29.02655124, 29.98755053, 35.8526023 , 37.30922154,
        39.34571186, 39.83318307, 41.60732638, 41.87218617, 42.70354741]]), array([[     8, 130489,  28936, 118371,  85260, 117954, 125095,  42643,
         92566,  30373,  58859,  84914,  37531,  74275,  32030]]))
----------------------------


In [7]:
spotify.iloc[130489]

Unnamed: 0                          130489
index                               130489
artist_name               ShooterGang Kony
track_id            1aEvsjCo142yxn02ACMCd7
track_name                       Skoompin'
acousticness                         0.148
danceability                         0.772
duration_ms                         158496
energy                               0.623
instrumentalness                         0
key                                      7
liveness                             0.408
loudness                            -6.149
mode                                     1
speechiness                          0.354
tempo                              102.953
time_signature                           4
valence                              0.692
popularity                              53
genre                                  rap
Name: 130489, dtype: object

In [8]:
spotify.iloc[8]

Unnamed: 0                               8
index                                    8
artist_name                             YG
track_id            0SDrD96V1uEJLptciyML9s
track_name                        10 TIMES
acousticness                         0.115
danceability                         0.841
duration_ms                         158493
energy                               0.502
instrumentalness                   0.00039
key                                      1
liveness                             0.591
loudness                            -7.744
mode                                     0
speechiness                         0.0394
tempo                              103.002
time_signature                           4
valence                              0.169
popularity                              60
genre                              hip-hop
Name: 8, dtype: object

# **Song Suggestions**

In [9]:
"""
Using for loops to pull the Suggestions of 1 through 5, then add them into a
list to be used for pulling the ID's to be put into the database.
"""

suggestions1 = []

for index in range(len(train)):
    prediction = nn.kneighbors([train.iloc[index].values])
    preds = prediction[1]
    prd = preds[0,1]
    suggestions1.insert(index, prd)

In [10]:
suggestions2 = []

for index in range(len(train)):
    prediction = nn.kneighbors([train.iloc[index].values])
    preds = prediction[1]
    prd = preds[0,2]
    suggestions2.insert(index, prd)

In [11]:
suggestions3 = []

for index in range(len(train)):
    prediction = nn.kneighbors([train.iloc[index].values])
    preds = prediction[1]
    prd = preds[0,3]
    suggestions3.insert(index, prd)

In [12]:
suggestions4 = []

for index in range(len(train)):
    prediction = nn.kneighbors([train.iloc[index].values])
    preds = prediction[1]
    prd = preds[0,4]
    suggestions4.insert(index, prd)

In [13]:
suggestions5 = []

for index in range(len(train)):
    prediction = nn.kneighbors([train.iloc[index].values])
    preds = prediction[1]
    prd = preds[0,5]
    suggestions5.insert(index, prd)

In [14]:
"""
Resetting the index so that I can pull the song id information
to combine with the suggestion information.
"""
train = train.reset_index()

In [15]:
"""
For loops to use the list of indices to pull the ids for suggestions then
inserting them into the list at the correct index, for each suggested song
to then be added into the database.
"""
ids1 = []
for index in range(len(train)):
  ids1.insert(index, train['track_id'][suggestions1[index]])

In [16]:
ids2 = []
for index in range(len(train)):
  ids2.insert(index, train['track_id'][suggestions2[index]])

In [17]:
ids3 = []
for index in range(len(train)):
  ids3.insert(index, train['track_id'][suggestions3[index]])

In [18]:
ids4 = []
for index in range(len(train)):
  ids4.insert(index, train['track_id'][suggestions4[index]])

In [19]:
ids5 = []
for index in range(len(train)):
  ids5.insert(index, train['track_id'][suggestions5[index]])

In [20]:
"""
Adding all the suggested song ids into the database with the original
song.
"""
train['suggested_id_1']= ids1
train['suggested_id_2']= ids2
train['suggested_id_3']= ids3
train['suggested_id_4']= ids4
train['suggested_id_5']= ids5

# **Mood Function**

In [21]:
def mood(X):
  """
  A Function that takes the rating from the Spotify
  API for Danceability, Liveness, Valence, Energy,
  Tempo, Speechiness, Instrumentalness, and Accousticness; 
  then depending upon that rating, returns either Low, Med, or
  High. This is to be put with the Song Suggestions for the Web
  Backend to connection into the DS API, so that the user can query
  mood as well as song.
  """
    X = X.copy()

    dance = []
    for row in range(len(X.danceability)):
        if X.danceability[row] <= .300:
            dance.insert(row, 'Low')
        elif X.danceability[row] >= .600:
            dance.insert(row, 'High')
        else:
            dance.insert(row, 'Med')

    
    intimate = []
    for row in range(len(X.liveness)):
        if X.liveness[row] <= .300:
            intimate.insert(row, 'Low')
        elif X.liveness[row] >= .600:
            intimate.insert(row, 'High')
        else:
            intimate.insert(row, 'Med')


    happy = []
    for row in range(len(X.valence)):
        if X.valence[row] <= .300:
            happy.insert(row, 'Low')
        elif X.valence[row] >= .600:
            happy.insert(row, 'High')
        else:
            happy.insert(row, 'Med')


    energys = []
    for row in range(len(X.energy)):
        if X.energy[row] <= .300:
            energys.insert(row, 'Low')
        elif X.energy[row] >= .600:
            energys.insert(row, 'High')
        else:
            energys.insert(row, 'Med')


    tempos = []
    for row in range(len(X.tempo)):
        if X.tempo[row] <= .300:
            tempos.insert(row, 'Low')
        elif X.tempo[row] >= .600:
            tempos.insert(row, 'High')
        else:
            tempos.insert(row, 'Med')
    

    lyrics = []
    for row in range(len(X.speechiness)):
        if X.speechiness[row] <= .300:
            lyrics.insert(row, 'Low')
        elif X.speechiness[row] >= .600:
            lyrics.insert(row, 'High')
        else:
            lyrics.insert(row, 'Med')


    instrument = []
    for row in range(len(X.instrumentalness)):
        if X.instrumentalness[row] <= .300:
            instrument.insert(row, 'Low')
        elif X.instrumentalness[row] >= .600:
            instrument.insert(row, 'High')
        else:
            instrument.insert(row, 'Med')


    accoustic = []
    for row in range(len(X.acousticness)):
        if X.acousticness[row] <= .300:
            accoustic.insert(row, 'Low')
        elif X.acousticness[row] >= .600:
            accoustic.insert(row, 'High')
        else:
            accoustic.insert(row, 'Med')

    
    X['Dance'] = dance
    X['Intimate'] = intimate
    X['Happy'] = happy
    X['Energy'] = energys
    X['Tempos'] = tempos
    X['Lyrics'] = lyrics
    X['Instrument'] = instrument
    X['Accoustic'] = accoustic

    return X

In [22]:
train = mood(train)

# **Final CSV**

In [23]:
"""
Using the current columns to remove the columns I want to keep.
Keeping the low, med, high mood ratings, the original track id,
and the top 5 suggested songs for that song based off of the
nearest neighbors model
"""
columns = train.columns
columns_keep = ['suggested_id_1', 'suggested_id_2', 'suggested_id_3','suggested_id_4','suggested_id_5', 'Dance',
       'Intimate', 'Happy', 'Tempos', 'Energy','Lyrics', 'Instrument', 'Accoustic', 'track_id']

columns = columns.drop(columns_keep)
train = train.drop(columns=columns)

In [24]:
train.head()

,track_id,suggested_id_1,suggested_id_2,suggested_id_3,suggested_id_4,suggested_id_5,Dance,Intimate,Happy,Energy,Tempos,Lyrics,Instrument,Accoustic
0,2RM4jf1Xa9zPgMGRDiht8O,6ZbQJplIhznkai9Duxv4BB,5W12joIh0XrwMkQScZclw5,6OxiZ2AKy9UFAmdumLYnTf,6XzyAQs2jU4PMWmnFlEQLW,0sEK0sQ2hlJYAuA3bnxrAn,High,Low,Low,Med,High,Med,Low,Low
1,1tHDG53xJNGsItRA3vfVgs,0go0d980ZLJk5WsR9eXR4s,2k29JB9QSXg4GF46LjhM3X,4lRO89kqUc97CWtil3e8Uz,2jdUcTkiAd2OVr9NmBPWd4,1GWLMDEwCeVELrW3qphA9a,High,Low,Med,Med,High,Med,Low,Low
2,1pWeQDyW5ieMmfjr8HP5TU,50XL9fJ99S5LPRLnXSsiOs,4cb2DSkJVDWcMOPgXpEWB6,0LHCGhmrr3M94fS5B4oqa7,0Rt6sKoyTpgywEceQlOm4I,70aGjUBECmba2TGCETwxNW,High,Low,Low,Med,High,Med,Low,Low
3,75NGdzR6LSC1r0KB5mltP9,51HNLw4bc3vK33IF0M4PDy,4fZRm3sCdRAInQ2nH134Ej,2BbUQfI5pp4wa5FrcRLRUE,2oCjNHAJfXQ9Ga70n67Nzb,4tXDGIPLRuXn4Q2UBi9TU7,High,Low,Med,Med,High,Low,Low,Low
4,3AT4akM1Oj1dfCb0ZA9cdy,2lRe5wBRm4xaSKTbDn2vLD,0KDxmYBQUw16diadSah97A,739WqqewmLt5IdBM9JaLjs,2CAegssA9BhQKSsJiSoWCS,0X2sc95uuQRPbMLbPZ2aOv,High,Low,Low,Med,High,Med,Low,Low


In [25]:
train.to_csv('large_song_data.csv')

# **Visualizations**

In [4]:
import plotly.graph_objects as go
import numpy as np

title = 'Favorite Vs. Suggested'
labels = ['Song', 'pred1', 'pred2', 'pred3','pred4', 'pred5']
colors = ['rgb(67,67,67)', 'rgb(115,115,115)', 'rgb(115,115,115)', 'rgb(115,115,115)',
          'rgb(115,115,115)', 'rgb(115,115,115)']
xlabels = ['Accousticness', 'Danceability', 'Energy',
               'Instrumentalness', 'Liveness', 'Speechiness',
               'Tempo', 'Valence']
mode_size = [8, 8, 12, 8]
line_size = [2, 2, 4, 2]

x_data = np.array([['Accousticness', 'Danceability', 'Energy',
               'Instrumentalness', 'Liveness', 'Speechiness',
               'Tempo', 'Valence']])
                  

y_data = np.array([
                   [0.115, 0.841, 0.502, 0.00039, 0.591, 0.0394, .103, 0.169],
                   [0.566, 0.808, 0.451, 0, 0.1, 0.0706, .096, 0.528],
                   [0.148, 0.772, 0.623, 0, 0.408, 0.354, .102, 0.692],
                   [0.692, 0.753, 0.708, 0, 0.103, 0.0316, .115, 0.794],
                   [0.00152, 0.664, 0.86, 0.894, 0.197, 0.0345, .121, 0.249],
                   [0.876, 0.271, 0.128, 0.000245, 0.11, 0.0403, .084, 0.127]
])

fig = go.Figure()

for i in range(0, 6):
    fig.add_trace(go.Scatter(x=x_data[0], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[1]),
        connectgaps=True,
    ))


fig.update_layout(
    xaxis=dict(
        ticktext=xlabels,
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False),

    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the left_side of the plot
    annotations.append(dict(xref='paper', x=0.05, y=y_trace[0],
                                  xanchor='right', yanchor='middle',
                                  text=label,
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
    
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.15,
                              xanchor='left', yanchor='bottom',
                              text='Song Vs. Predictions',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

fig.update_layout(annotations=annotations)

fig.show()

In [5]:
# this is importing
gdrive_file_id = '1grunsDCYl4Ro67EuDiFfcWXMBoAhrY6J'
df = pd.read_csv(f'https://docs.google.com/uc?id={gdrive_file_id}&export=download', encoding='ISO-8859-1',index_col=0)
df['tempo'] = df['tempo'] * 0.001
#---------seperate above for loading times
track_id = '2RM4jf1Xa9zPgMGRDiht8O'
ogtrack = df[df['track_id'].str.match(track_id)]
sug1track = df[df['track_id'].str.match(ogtrack['suggested_id_1'][0])]
sug2track = df[df['track_id'].str.match(ogtrack['suggested_id_2'][0])]
sug3track = df[df['track_id'].str.match(ogtrack['suggested_id_3'][0])]
sug4track = df[df['track_id'].str.match(ogtrack['suggested_id_4'][0])]
sug5track = df[df['track_id'].str.match(ogtrack['suggested_id_5'][0])]
# --------graph--------------

title = 'Favorite Vs. Suggested'
labels = [ogtrack.iloc[-1]['track_name'], sug1track.iloc[-1]['track_name'], sug2track.iloc[-1]['track_name'], sug3track.iloc[-1]['track_name'],sug4track.iloc[-1]['track_name'], sug5track.iloc[-1]['track_name']]
colors = ['#d62728','#1f77b4', '#e377c2', '#17becf',
          'rgb(115,115,115)', '#2ca02c']
xlabels = ['Accousticness', 'Danceability', 'Energy',
               'Instrumentalness', 'Liveness', 'Speechiness',
               'Tempo', 'Valence']
mode_size = [8, 8, 12, 8]
line_size = [3, 1, 1, 1, 1, 1]
x_data = np.array([['Accousticness', 'Danceability', 'Energy',
               'Instrumentalness', 'Liveness', 'Speechiness',
               'Tempo', 'Valence']])
y_data = np.array([
                   [ogtrack.iloc[-1]['acousticness'], ogtrack.iloc[-1]['danceability'], ogtrack.iloc[-1]['energy'], ogtrack.iloc[-1]['instrumentalness'], ogtrack.iloc[-1]['liveness'], ogtrack.iloc[-1]['speechiness'], ogtrack.iloc[-1]['tempo'], ogtrack.iloc[-1]['valence']],
                   [sug1track.iloc[-1]['acousticness'], sug1track.iloc[-1]['danceability'], sug1track.iloc[-1]['energy'], sug1track.iloc[-1]['instrumentalness'], sug1track.iloc[-1]['liveness'], sug1track.iloc[-1]['speechiness'], sug1track.iloc[-1]['tempo'], sug1track.iloc[-1]['valence']],
                   [sug2track.iloc[-1]['acousticness'], sug2track.iloc[-1]['danceability'], sug2track.iloc[-1]['energy'], sug2track.iloc[-1]['instrumentalness'], sug2track.iloc[-1]['liveness'], sug2track.iloc[-1]['speechiness'], sug2track.iloc[-1]['tempo'], sug2track.iloc[-1]['valence']],
                   [sug3track.iloc[-1]['acousticness'], sug3track.iloc[-1]['danceability'], sug3track.iloc[-1]['energy'], sug3track.iloc[-1]['instrumentalness'], sug3track.iloc[-1]['liveness'], sug3track.iloc[-1]['speechiness'], sug3track.iloc[-1]['tempo'], sug3track.iloc[-1]['valence']],
                   [sug4track.iloc[-1]['acousticness'], sug4track.iloc[-1]['danceability'], sug4track.iloc[-1]['energy'], sug4track.iloc[-1]['instrumentalness'], sug4track.iloc[-1]['liveness'], sug4track.iloc[-1]['speechiness'], sug4track.iloc[-1]['tempo'], sug4track.iloc[-1]['valence']],
                   [sug5track.iloc[-1]['acousticness'], sug5track.iloc[-1]['danceability'], sug5track.iloc[-1]['energy'], sug5track.iloc[-1]['instrumentalness'], sug5track.iloc[-1]['liveness'], sug5track.iloc[-1]['speechiness'], sug5track.iloc[-1]['tempo'], sug5track.iloc[-1]['valence']]
])
fig = go.Figure()

for i in range(0, 6):
    fig.add_trace(go.Scatter(x=x_data[0], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))


fig.update_layout(
    xaxis=dict(
        ticktext=xlabels,
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False),

    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white',
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=.9,
    xanchor="right",
    x=1.05)
)

annotations = []
    
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.2,
                              xanchor='left', yanchor='bottom',
                              text='Song Vs. Suggestions',
                              font=dict(family='Arial',
                                        size=24,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

fig.update_layout(annotations=annotations)


fig.show()